# catboost model
sample of catboost model.  
local validation score is 0.3063986193670301.  
this result is not good, but catboost don't have to feature engineering.
so use for baseline model and ensemble with another model.

# import modules

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import os
from sklearn.metrics import f1_score

# load data

In [2]:
SEED = 314
datasrc = "data/official/"
data = pd.read_csv(os.path.join(datasrc, "train.csv"), index_col=0)
x_test = pd.read_csv(os.path.join(datasrc, "test.csv"), index_col=0)

train, valid = train_test_split(data, test_size=0.2, random_state=42, stratify=data["health"])

In [3]:
x_train = train.drop("health", axis=1)
y_train = train["health"]
x_valid = valid.drop("health", axis=1)
y_valid = valid["health"]

In [4]:
x_train.head()

,created_at,tree_dbh,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,spc_latin,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
2622,2016-06-10,3,OnCurb,1or2,Helpful,NoDamage,Volunteer,RootOtherTrunkOtherBranchOther,American elm,Ulmus americana,MN03,Central Harlem North-Polo Grounds,1,1021200,Manhattan,New York,110,30,70,9
13191,2015-11-04,6,OnCurb,NaN,NaN,Damage,Volunteer,NaN,pin oak,Quercus palustris,QN30,Maspeth,4,4052100,Queens,Maspeth,405,15,30,30
16871,2015-07-28,29,OnCurb,NaN,NaN,Damage,TreesCount Staff,NaN,red maple,Acer rubrum,SI28,Port Richmond,5,5021300,Staten Island,Staten Island,501,23,61,49
13477,2015-11-11,9,OnCurb,NaN,NaN,Damage,TreesCount Staff,StonesBranchLights,Callery pear,Pyrus calleryana,MN22,East Village,1,1003800,Manhattan,New York,103,27,66,2
10865,2015-10-28,32,OnCurb,NaN,NaN,NoDamage,TreesCount Staff,NaN,pin oak,Quercus palustris,SI11,Charleston-Richmond Valley-Tottenville,5,5022600,Staten Island,Staten Island,503,24,62,51


# preprocess data

In [5]:
x_train.loc[:,'created_at'] = pd.to_datetime(x_train.loc[:,'created_at'])
x_valid.loc[:,'created_at'] = pd.to_datetime(x_valid.loc[:,'created_at'])
x_test.loc[:,'created_at'] = pd.to_datetime(x_test.loc[:,'created_at'])

x_train = x_train.fillna("Dropped")
x_valid = x_valid.fillna("Dropped")
x_test = x_test.fillna("Dropped")

In [6]:
x_train.loc[:,'month'] = x_train.loc[:,'created_at'].dt.month
x_valid.loc[:,'month'] = x_valid.loc[:,'created_at'].dt.month
x_test.loc[:,'month'] = x_test.loc[:,'created_at'].dt.month

x_train.loc[:,'day'] = x_train.loc[:,'created_at'].dt.day
x_valid.loc[:,'day'] = x_valid.loc[:,'created_at'].dt.day
x_test.loc[:,'day'] = x_test.loc[:,'created_at'].dt.day

x_train.loc[:,'month_day_sin'] = np.sin(2 * np.pi * x_train.loc[:,'month'] / 12 + 2 * np.pi * x_train.loc[:,'day'] / 31)
x_valid.loc[:,'month_day_sin'] = np.sin(2 * np.pi * x_valid.loc[:,'month'] / 12 + 2 * np.pi * x_valid.loc[:,'day'] / 31)
x_test.loc[:,'month_day_sin'] = np.sin(2 * np.pi * x_test.loc[:,'month'] / 12 + 2 * np.pi * x_test.loc[:,'day'] / 31)

x_train.loc[:,'month_day_cos'] = np.cos(2 * np.pi * x_train.loc[:,'month'] / 12 + 2 * np.pi * x_train.loc[:,'day'] / 31)
x_valid.loc[:,'month_day_cos'] = np.cos(2 * np.pi * x_valid.loc[:,'month'] / 12 + 2 * np.pi * x_valid.loc[:,'day'] / 31)
x_test.loc[:,'month_day_cos'] = np.cos(2 * np.pi * x_test.loc[:,'month'] / 12 + 2 * np.pi * x_test.loc[:,'day'] / 31)

In [7]:
ignore_columns = [
    "created_at",
    "month",
    "day",
    "problems",
    "nta_name",
    "boro_ct",
    "boroname",
    "zip_city",
    "cb_num",
    "st_senate",
    "st_assem",
    "cncldist",
]

x_train = x_train.drop(ignore_columns, axis=1)
x_valid = x_valid.drop(ignore_columns, axis=1)
x_test = x_test.drop(ignore_columns, axis=1)

In [8]:
object_columns = x_train.select_dtypes(include=["object"]).columns.tolist()
print(object_columns)

['curb_loc', 'steward', 'guards', 'sidewalk', 'user_type', 'spc_common', 'spc_latin', 'nta']


# train model

In [9]:
model = CatBoostClassifier(
    cat_features=object_columns,
    iterations=214000, # 3000 < 29000 < 50000 < 100000 < 214000 is best
    early_stopping_rounds=3000,
    eval_metric='TotalF1',
    random_seed=SEED,
    learning_rate=0.001,
    max_depth=10,
    od_type='Iter',
    task_type="GPU",
    devices='0:1',
    verbose=1000,
    )

In [10]:
model.fit(x_train, y_train)

0:	learn: 0.6992220	total: 35.9ms	remaining: 2h 8m 12s
1000:	learn: 0.6948471	total: 36.3s	remaining: 2h 8m 42s
2000:	learn: 0.6952950	total: 1m 10s	remaining: 2h 5m 1s
3000:	learn: 0.6958909	total: 1m 43s	remaining: 2h 1m 23s
4000:	learn: 0.6958909	total: 2m 17s	remaining: 2h 8s
5000:	learn: 0.6976703	total: 2m 51s	remaining: 1h 59m 33s
6000:	learn: 0.6988498	total: 3m 26s	remaining: 1h 59m 24s
7000:	learn: 0.6995842	total: 4m 1s	remaining: 1h 58m 59s
8000:	learn: 0.7011926	total: 4m 36s	remaining: 1h 58m 31s
9000:	learn: 0.7023602	total: 5m 11s	remaining: 1h 58m 19s
10000:	learn: 0.7058318	total: 5m 46s	remaining: 1h 57m 56s
11000:	learn: 0.7089795	total: 6m 21s	remaining: 1h 57m 25s
12000:	learn: 0.7115324	total: 6m 56s	remaining: 1h 56m 56s
13000:	learn: 0.7144938	total: 7m 31s	remaining: 1h 56m 20s
14000:	learn: 0.7167307	total: 8m 6s	remaining: 1h 55m 47s
15000:	learn: 0.7207415	total: 8m 41s	remaining: 1h 55m 16s
16000:	learn: 0.7241652	total: 9m 16s	remaining: 1h 54m 42s
17000:

# predict

In [11]:
y_pred = model.predict(x_valid)
y_prob = model.predict_proba(x_valid)

# compare prediction vs actual

In [18]:
y_train.value_counts(normalize=True)

health
1    0.788203
0    0.176894
2    0.034903
Name: proportion, dtype: float64

In [12]:
pd.DataFrame(y_pred).value_counts(normalize=True)

1    0.974481
0    0.025019
2    0.000500
Name: proportion, dtype: float64

# calculate score

In [13]:
f1_score(y_valid, y_pred, average='macro')

0.3063986193670301

# maximize score

In [14]:
# this model's maximize f1 score
def sort_predict(model, x, ratios=[0.176, 0.788, 0.035]):
    pred = model.predict_proba(x)
    pred = pd.DataFrame(pred, columns=["good", "normal", "bad"], index=x.index)
    good_count = int(len(x)*ratios[0])
    # normal_count = int(len(x)*ratios[1])
    bad_count = int(len(x)*ratios[2])
    pred_sortbybad = pred.sort_values(by="bad", ascending=False)
    # pred_sortbynormal = pred.sort_values(by="normal", ascending=False)
    pred_sortbygood = pred.sort_values(by="good", ascending=False)
    pred["health"] = 1
    pred.loc[pred_sortbybad.iloc[:bad_count, :].index, "health"] = 2
    pred.loc[pred_sortbygood.iloc[:good_count, :].index, "health"] = 0
    return pred["health"]

ratio = train["health"].value_counts(normalize=True).sort_index()
f1_score(y_valid, sort_predict(model, x_valid, ratio), average="macro")

0.33836040138968304

# analyze model

In [15]:
model.get_feature_importance(
    prettified=True,
    type="PredictionValuesChange",
    )

,Feature Id,Importances
0,nta,20.906735
1,spc_common,18.509285
2,tree_dbh,11.169107
3,month_day_cos,9.822904
4,month_day_sin,9.762655
5,user_type,8.166497
6,borocode,7.733324
7,sidewalk,6.119985
8,guards,5.409644
9,steward,1.935396


# make submission

In [16]:
y_test = model.predict(x_test)

In [17]:
submission = pd.DataFrame(y_test, index=x_test.index, columns=["health"])
submission.to_csv("submission/catboost_submission.csv", header=False)